"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
### Alias : proto.ipynb & Last Modded : 2023.07.09. ###
Coded with Python 3.10 Grammar
Description : FOS Controller Prototype
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

#### Basic Settings

In [1]:
import asyncio
import socket
import json

import pandas as pd

HOST = "127.0.0.1"
PORT = 8080
SIZE = 4096

In [2]:
def recv():
    global client_socket
    fragments = []
    while True:
        chunk = client_socket.recv(SIZE)
        if not chunk:
            raise Exception("Socket connection closed unexpectedly.")
        fragments.append(chunk)
        if len(chunk) < SIZE:
            break
    return b''.join(fragments).decode()

#### Server Open

In [3]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen()

#### Client Accept

In [5]:
client_socket, client_addr = server_socket.accept()

In [6]:
msg = recv()
msg

'!CONNECTED!'

#### Game Start

In [7]:
client_socket.sendall("/start".encode())

In [8]:
msg = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
msg

,0
code,200
message,Round 1 is ready.
data,{'round': 1}


In [9]:
client_socket.sendall("/data/aircraft_specsheet".encode())

In [10]:
spec = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
spec

,0
code,200
message,Success
data,"{""D1"": {""Aircraft Type"": ""Drone"", ""Velocity"": ..."


In [12]:
pd.DataFrame.from_dict(data=json.loads(spec[0]['data']), orient='index')

,Aircraft Type,Velocity,ETRDY,Cost,Cover Area,Water Tank,Possibility
D1,Drone,80,10,500,0,7.2,56
D2,Drone,100,12,600,0,6.8,57
D3,Drone,50,8,400,0,7.5,55
D4,Drone,90,10,550,0,6.7,57
D5,Drone,70,9,450,0,7.4,56
H1,Helicopter,200,15,1000,1,6.5,61
H2,Helicopter,190,14,950,1,6.8,62
H3,Helicopter,210,16,1100,1,6.1,63
H4,Helicopter,195,14,900,1,6.7,60
H5,Helicopter,205,15,1050,1,6.3,62


In [13]:
client_socket.sendall("/data/target_list".encode())

In [14]:
targets = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
targets

,0
code,200
message,Success
data,"{""Targets"": {""T1"": {""latitude"": 280, ""longitud..."


In [15]:
pd.DataFrame.from_dict(data=json.loads(targets[0]['data']), orient='columns')

,Targets,Lakes,Bases
T1,"{'latitude': 280, 'longitude': 20, 'targeted':...",NaN,NaN
T2,"{'latitude': 280, 'longitude': 60, 'targeted':...",NaN,NaN
T3,"{'latitude': 280, 'longitude': 100, 'targeted'...",NaN,NaN
T4,"{'latitude': 240, 'longitude': 20, 'targeted':...",NaN,NaN
T5,"{'latitude': 240, 'longitude': 60, 'targeted':...",NaN,NaN
T6,"{'latitude': 240, 'longitude': 100, 'targeted'...",NaN,NaN
T7,"{'latitude': 200, 'longitude': 20, 'targeted':...",NaN,NaN
T8,"{'latitude': 200, 'longitude': 60, 'targeted':...",NaN,NaN
T9,"{'latitude': 200, 'longitude': 100, 'targeted'...",NaN,NaN
L1,NaN,"{'latitude': 270, 'longitude': 270}",NaN


In [16]:
client_socket.sendall("/data/unit_table".encode())

In [17]:
units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
units

,0
code,200
message,Success
data,"{""D1-A"": {""Ordered"": false, ""Available"": true,..."


In [18]:
pd.DataFrame.from_dict(data=json.loads(units[0]['data']), orient='index')

,Ordered,Available,ETR,ETD,ETA,Base,Current Water
D1-A,False,True,,,,A,50.888889
D2-A,False,True,,,,C,50.705882
D3-A,False,True,,,,C,99.333333
D4-A,False,True,,,,C,77.925373
D5-A,False,True,,,,A,34.513514
H1-A,False,True,,,,C,44.384615
H2-A,False,True,,,,C,33.705882
H3-A,False,True,,,,B,33.393443
H4-A,False,True,,,,C,18.925373
H5-A,False,True,,,,C,16.873016


In [19]:
xml = """<operations>
    <order>
        <time>0601</time>
        <base>A</base>
        <aircraft_type>Airplane</aircraft_type>
        <track_number>A4-B</track_number>
        <mission_type>1</mission_type>
        <course>T8</course>
    </order>
    <order>
        <time>0602</time>
        <base>B</base>
        <aircraft_type>Drone</aircraft_type>
        <track_number>D1-A</track_number>
        <mission_type>1</mission_type>
        <course>T8</course>
    </order>
</operations>"""

In [20]:
client_socket.sendall(f"/order/{xml}".encode())

In [21]:
order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
order_result

,0
code,500
message,'aircraft_id'
